% CGRtools Tutorial
% Dr. Ramil Nugmanov; Dr. Timur Madzhidov; Ravil Mukhametgaleev
% Mar 29, 2020

# 1. Data types and operations with them

(c) 2020, Dr. Ramil Nugmanov; Dr. Timur Madzhidov; Ravil Mukhametgaleev

Installation instructions of CGRtools package information and tutorial's files see on `https://github.com/cimm-kzn/CGRtools`

NOTE: Tutorial should be performed sequentially from the start. Random cell running will lead to unexpected results. 

In [1]:
import pkg_resources
if pkg_resources.get_distribution('CGRtools').version.split('.')[:2] != ['4', '0']:
    print('WARNING. Tutorial was tested on 4.0 version of CGRtools')
else:
    print('Welcome!')

Welcome!


In [2]:
# load data for tutorial
from pickle import load
from traceback import format_exc

with open('molecules.dat', 'rb') as f:
    molecules = load(f) # list of MoleculeContainer objects
with open('reactions.dat', 'rb') as f:
    reactions = load(f) # list of ReactionContainer objects

m1, m2, m3, m4 = molecules # molecule
m7 = m3.copy()
m11 = m3.copy()
m11.standardize()
m7.standardize()
r1 = reactions[0] # reaction
m5 = r1.reactants[0]
m8 = m7.substructure([4, 5, 6, 7, 8, 9])
m10 =  r1.products[0].copy()

CGRtools has subpackage *containers* with data structures classes:

* *MoleculeContainer* - for molecular structure
* *ReactionContainer* - for chemical reaction 
* *CGRContainer* - for Condensed Graph of Reaction
* *QueryContainer* - queries for substructure search in molecules
* *QueryCGRContainer* - queries for substructure search in CGRs

In [3]:
# import all containers

from CGRtools.containers import * 

## 1.1. MoleculeContainer
Molecules are represented as undirected graphs. Molecules contain *Atom* objects and *Bond* objects.

*Atom* objects are represented as dictionary with unique number for each atom as key.  

*Bond* objects are stored as sparse matrix with adjacent atoms pair as keys for rows and columns.

Hereafter, *atom number* is unique integer used to enumerate atoms in molecule. Please, don't confuse it with element number in Periodic Table, hereafter called *element number*.

Methods for molecule handling and the arguments of *MoleculeContainer* are described below.

In [4]:
# dictionary for molecule properties storage. 
# For example, DTYPE/DATUM fields of SDF file are read into this dictionary

m1.meta 

{'cdid': '001'}

In [5]:
# MoleculeContainer supports depiction and graphic representation in Jupyter notebooks.

m1 

In [6]:
# depiction returns SVG image in format string

m1.depict()

'<svg width="2.18cm" height="1.04cm" viewBox="-11.13 -0.45 2.18 1.04" xmlns="http://www.w3.org/2000/svg" version="1.1">\n  <defs>\n    <mask id="mask-1f35e106-e3e7-433a-b416-05609af1bc73">\n      <rect x="-11.13" y="-0.45" width="2.18" height="1.04" fill="white"/>\n      <g fill="black">\n        <circle cx="-9.38" cy="-0.14" r="0.12"/>\n      </g>\n    </mask>\n  </defs>\n  <g fill="none" stroke="black" stroke-width="0.03"  mask="url(#mask-1f35e106-e3e7-433a-b416-05609af1bc73)">\n    <line x1="-10.79" y1="-0.17" x2="-10.08" y2="0.24"/>\n    <line x1="-10.83" y1="-0.10" x2="-10.12" y2="0.31"/>\n    <line x1="-10.10" y1="0.28" x2="-9.38" y2="-0.14"/>\n  </g>\n  <g font-family="monospace">\n    <g fill="#FF0D0D">\n      <text x="-9.38" y="-0.14" dx="-0.07" dy="0.07" font-size="0.25">OH</text>\n      <text x="-9.38" y="-0.14" dx="-0.07" dy="-0.05" font-size="0.15" text-anchor="end">15</text>\n    </g>\n    <g fill="#788CFF" font-size="0.20">\n      <text x="-10.81" y="-0.14" text-anchor="

In [7]:
# saving image to SVG file

with open('molecule.svg', 'w') as f: 
    f.write(m1.depict())

In [8]:
# isolated copy of molecule

m_copy = m1.copy() 
m_copy

In [9]:
# get number of atoms in molecule
print(len(m1))

# or 
m1.atoms_count

3


3

In [10]:
# number of bonds

m1.bonds_count 

2

In [11]:
# list of atoms numbers

m1.atoms_numbers 

(3, 4, 1)

The following notations are used for hybridization of atoms. Values are given as numbers below (in parenthesis symbols that are used in SMILES-like signatures are shown):

* 1 (s) - all bonds of atom are single, i.e. sp3 hybridization
* 2 (d) - atom has one double bond and others are single, i.e. sp2 hybridization
* 3 (t) - atom has one triple or two double bonds and other are single, i.e. sp hybridization
* 4 (a) - atom is in aromatic ring

Neighbors and hybridizations atom attributes are required for **substructure operations** and structure standardization. See below

In [12]:
# iterate over atoms using its numbers

list(m1.atoms())  # works the same as dict.items()

[(3, C()), (4, C()), (1, O(15))]

In [13]:
# iterate over bonds using adjacent atoms numbers

list(m1.bonds())

[(3, 4, Bond(2)), (4, 1, Bond(1))]

In [14]:
# access to atom by number

m1.atom(1)

O(15)

In [15]:
try:
    m1.atom(10) # raise error for absent atom numbers
except KeyError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-15-3121e0ab5f15>", line 2, in <module>
    m1.atom(10) # raise error for absent atom numbers
  File "/Users/teg/PycharmProjects/CGRtools/CGRtools/containers/common.py", line 79, in atom
    return self._atoms[n]
KeyError: 10



In [16]:
# access to bond using adjacent atoms numbers
m1.bond(1, 4)

Bond(1)

In [17]:
try:
    m1.bond(1, 3) # raise error for absent bond
except KeyError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-17-e981e9e02512>", line 2, in <module>
    m1.bond(1, 3) # raise error for absent bond
  File "/Users/teg/PycharmProjects/CGRtools/CGRtools/containers/common.py", line 109, in bond
    return self._bonds[n][m]
KeyError: 3



#### Atom objects are dictionary-like classes which store information about:
* atomic symbol (element)
* atomic number
* atomic radius
* isotope (atomic mass)
* charge
* valence_rules
* xyz coordinates

Also atoms has methods for data integrity checks and include some internally used data.

In [18]:
a = m1.atom(1)

# access to information
# element symbol
a.atomic_symbol

'O'

In [19]:
# atomic number

a.atomic_number

8

In [20]:
# atomic radius

a.atomic_radius

0.48

In [21]:
# atom isotope. Default isotope if not set. 
# Default isotopes are the same as used in InChI notation

a.isotope 

15

In [22]:
# formal charge

a.charge 

0

In [23]:
# coordinates

print(a.x) 
print(a.y)
print(a.xy)

-9.3843
0.1375
(-9.3843, 0.1375)


In [24]:
# Number of neighboring atoms. It is read-only.

a.neighbors

1

In [25]:
# Atoms hybridization. It is read-only.

a.hybridization 

1

In [26]:
try:
    a.hybridization = 2 # Not subsettable. Read-only! Thus error is raised.
except AttributeError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-26-5e03ec8155b6>", line 2, in <module>
    a.hybridization = 2 # Not subsettable. Read-only! Thus error is raised.
AttributeError: can't set attribute



In [27]:
# bond objects also are dictionary-like classes which store information about bond order
b = m1.bond(3, 4)

b.order
 # order change also possible

2

In [28]:
try:
    b.order = 1 # error raises. You can't set the order for created bond.
except AttributeError :
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-28-9d7c3ccaab2e>", line 2, in <module>
    b.order = 1 # error raises. You can't set the order for created bond.
AttributeError: can't set attribute



One should to use `delete_bond` method to break bond

In [29]:
m1.delete_bond(3, 4)
m1                    #Note that cashe flushing is not required here.

Method `delete_atom` removes atom from the molecule

In [30]:
m1.delete_atom(3)
m1

In [31]:
m_copy # copy unchanged!

*Atoms* and *bonds* objects can be converted into integer representation that could be used to classify their types.

*Atom* type is represented by 21 bit code rounded to 32 bit integer number:

* 7 bits stand for atom number (2 ** 7 - 1 == 127, currently 118 elements are presented in Periodic Table)
* 9 bits are used for isotope (511 posibilities, highest known isotope is ~300)
* 3 bits stand for formal charge. Charges range from -3 to +3 rescaled to range 0-6
* 2 bits are used for multiplicity.

In [32]:
int(a)
# 131596 == 0001000 000010000 011 00
# 0001000 == 8 Oxygen
# 000010000 == 16 isotope
# 011 == 3 (3 - 3 = 0) uncharged
# 00 == 0 hasn't multiplicity

61704

In [33]:
int(b)  # bonds are encoded by their order

2

In [34]:
m1

In [35]:
m1.check_valence() # return list of numbers of atoms with invalid valences

[]

In [36]:
m4 # molecule with valence errors

In [37]:
m4.check_valence()

[1]

In [38]:
m3

In [39]:
m3.sssr # Method for application of Smallest Set of Smallest Rings algorithm for rings 
        # identification. Returns list of lists of atoms forming smallest rings

((4, 9, 8, 7, 6, 5),)

#### Connected components.
Sometimes molecules has disconnected components (salts etc).

One can find them and split molecule to separate components.

In [40]:
m2 # it's a salt represented as one graph

In [41]:
m2.connected_components # list of lists of atoms belonging to graph components

((5, 8, 6, 7, 9, 10), (11,))

In [42]:
anion, cation = m2.split() # split molecule to components

In [43]:
anion # graph of only one salt component

In [44]:
cation # graph of only one salt component

#### Union of molecules
Sometimes it is more convenient to represent salts as ion pair. Otherwise ambiguity could be introduced, for example in reaction of salt exchange:

**Ag+** + **NO3-** + **Na+** + **Br-** = **Ag+** + **Br-** + **Na+** + **NO3-**. Reactants and products sets are the same. 

In this case one can combine anion-cation pair into single graph. It could be convenient way to represent other molecule mixtures.

In [45]:
salt = anion | cation 

# or 

anion.union(cation)

salt # this graph has disconnected components, it is considered single compound now

#### Substructures could be extracted from molecules.

By default, returned substructures are read-only projections of original molecule (except attributes of atoms/bonds). 

Changes in original molecule (bond breaking/formation, atom insertion/deletion, atom/bond attributes changes) will be mirrored in projection.  

Projections share the same neighbors and hybridization attributes as in initial molecule even if could be wrong for substructure 

In [46]:
proj = m3.substructure([4,5,6,7,8,9])  # projection of substructure
proj

In [47]:
m3.atom(4).neighbors

3

In [48]:
proj.atom(4).neighbors # same as in original molecule and not as it should be in substructure

2

In [49]:
benzene = m3.substructure([4,5,6,7,8,9]) # Substructure could be extracted as isolated graph (not projection)
benzene

In [50]:
benzene.atom(4).neighbors # now number of neighbors for atom 4 is 2. It is not 3 as above where projection was used.

2

Note:

* Projection of projection also projection of original molecule
* Projection can be converted to isolated molecule by calling method copy()

In [51]:
proj_copy = proj.copy() # turning projection into molecule using "copy" method
proj_copy

Changes in projection are mirrored. See example:

In [52]:
m3.delete_bond(4, 5) # we've deleted bond
m3

In [53]:
proj.flush_cache() # remove cached image. Note that in projection the bond is also deleted.
proj

`augmented_substructure` is a substructure consisting from atoms and a given number of shells of neighboring atoms around it.
**deep** argument is a number of considered shells. 

It also returns projection by default.

In [54]:
aug = m3.augmented_substructure([10], deep=2) #  atom 10 is Nitrogen
aug

In [55]:
aug.atom(10).hybridization #atom has two incident double bond.

3

#### Atoms Ordering.
This functionality is used for canonic numbering of atoms in molecules. Prime number multiplication based Morgan algorithm is used for atom ranking. Property `atoms_order` returns dictionary of atom numbers as keys and their ranks according to canonicalization as values. Equal rank mean that atoms are symmetric (are mapped to each other in automorhisms). In present version, instead of sequential ranks prime numbers are returned.


In [56]:
m5.atoms_order

{1: 1, 3: 2, 4: 3}

#### Atom number can be changed by `remap` method.

This method is useful when it is needed to change order of atoms in molecules. First argument to `remap` method is dictionary with existing atom numbers as keys and desired atom number as values. It is possible to change atom numbers for only part of atoms. Atom numbers could be non-sequencial but need to be unique. 

If argument *copy* is set ***True*** new object will be created, else existing molecule will be changed. Default is ***False***. 

In [57]:
for n, a in m5.atoms():
    print(n, a.atomic_symbol)
for n, m, b in m5.bonds():
    print(m, n, b.order)

3 C
4 C
1 O
4 3 1
1 4 1


In [58]:
remapped = m5.remap({4:2}, copy=True)
remapped

In [59]:
for n, a in remapped.atoms():
    print(n, a.atomic_symbol)
for n, m, b in remapped.bonds():
    print(m, n, b.order)

3 C
2 C
1 O
2 3 1
1 2 1


## 1.2. ReactionContainer

*ReactionContainer* objects has the following properties:

* **reactants** - list of reactants molecules
* **reagents** - list of reagents molecules
* **products** - list of products molecules
* **meta** - dictinary of reaction metadata (DTYPE/DATUM block in RDF)

In [60]:
r1 # depiction supported

In [61]:
r1.meta

{'CdId': '1872',
 'solvent': '3',
 'temperature': '129.5',
 'tabulated_constant': '-6.87'}

In [62]:
print(r1.reactants, r1.products)  # Access to lists of reactant and products. Molecules' signatures are returned by print() method.
reactant1, reactant2, reactant3 = r1.reactants
product = r1.products[0]

[<CGRtools.containers.molecule.MoleculeContainer object at 0x11e418c28>, <CGRtools.containers.molecule.MoleculeContainer object at 0x11e418cc8>, <CGRtools.containers.molecule.MoleculeContainer object at 0x11e418d68>] [<CGRtools.containers.molecule.MoleculeContainer object at 0x11e418e08>]


Reactions also has `standardize`, `aromatize`, `implicify_hydrogens` and `explicify_hydrogens` methods (see part 3). These methods are applied independently to every molecule in reaction.

## 1.3. CGR
*CGRContainer* object is similar to *MoleculeConrtainer*, except some methods. The following methods are not suppoted for *CGRContainer*:

* aromatize
* standardize
* implicify_hydrogens
* explicify_hydrogens
* atom_implicit_h
* atom_explicit_h
* atom_total_h
* check_valence

*CGRContainer* also has some methods absent in *MoleculeConrtainer*:

* centers_list
* center_atoms
* center_bonds

*CGRContainer* is undirected graph. Atoms and bonds in CGR has two states: reactant and product.

#### Composing to CGR

As mentioned above, atoms in *MoleculeContainer* have unique numbers. These numbers are used as atom-to-atom mapping in CGRtools upon CGR creation. Thus, atom order for molecules in reaction should correspond to atom-to-atom mapping.  

Pair of molecules can be transformed into CGR. Notice that, the same atom numbers in reagents and products imply the same atoms.

Reaction also can be composed into CGR. Atom numbers of molecules in reaction are used as atom-to-atom mapping of reactants to products.

In [63]:
cgr1 = m7 ^ m8 # CGR from molecules
# or 
m7.compose(m8)

print(cgr1)  # CGRcontainer object currently doesn't support depiction. CGR signature is printed out.

C=1(C=CC(=CC=1)[->.]O)[->.][N+]([O-])=O


This is CGR (depiction is made by ChemAxon externally). You can see changed bonds connected to ring.
![cgr1.png](cgr1.png)

In [64]:
r1

In [65]:
cgr2 = ~r1 # CGR from reactions

# or 

r1.compose()
print(cgr2) # signature is printed out.

CC[O->0][.>-]C(C([->.]O)([.>-]OCC)=O)([->.]O)=O


It is CGR for reaction (depiction is made by ChemAxon externally).
![cgr2.png](cgr2.png)


In [66]:
a = cgr2.atom(2) # atom access is the same as for MoleculeContainer

In [67]:
a.atomic_symbol # element attribute


'O'

In [68]:
a.isotope # isotope attribute


For *CGRContainer* attributes `charge`, `x`, `y`, `z`, `neighbors` and `hybridization` refer to atom state in reactant of reaction;  arguments `p_charge`, `p_multiplicity`, `p_x`, `p_y`, `p_z`, `p_neighbors` and `p_hybridization` could be used to extract atom state in product part in reaction.

In [69]:
a.charge # charge of atom in reactant

-1

In [70]:
a.p_charge # charge of atom in product

0

In [71]:
a.neighbors # number of neighbors of atom in reactant

1

In [72]:
a.p_neighbors # number of neighbors of atom in product

2

In [73]:
a.hybridization # hybridization of atom in reactant. 1 means only single bonds are incident to atom

1

In [74]:
a.p_hybridization # hybridization of atom in product. 1 means only single bonds are incident to atom

1

In [75]:
b = cgr1.bond(4, 10) # take bond

#### Bonds has `order` and `p_order` attribute
If `order` attribute value is ***None***, it means that bond was formed  
If `p_order` is ***None***, it means that bond was broken  

Both `order` and `p_order` can't be ***None***

In [76]:
b.order # bond order in reactant

1

In [77]:
b.p_order is None # bond order in product in None

True

**CGR can be decomposed back to reaction**, i.e. reactants and products.

Notice that CGR can lose information in case of unbalanced reactions (where some atoms of reactant does not have counterpart in product, and vice versa). Decomposition of CGRs for unbalanced reactions back to reaction may lead to strange (and erroneous) structures.

In [78]:
reactant_part, product_part = ~cgr1 # CGR of unbalanced reaction is decomposed back into reaction
# or 
cgr1.decompose()

(<CGRtools.containers.molecule.MoleculeContainer at 0x11e4afea8>,
 <CGRtools.containers.molecule.MoleculeContainer at 0x11e4afc28>)

In [79]:
reactant_part # reactants extracted. One can notice it is initial molecule

In [80]:
product_part #extracted products. Originally benzene was the product.

For decomposition of *CGRContainer* back into *ReactionContainer* `decompose` method can be used.

In [81]:
decomposed = cgr2.decompose() # decomposition of CGR2 into reaction

In [82]:
decomposed # You can see that water absent in products initially was restored. 
# This is a side-effect of CGR decomposing that could help with reaction balancing. 
# But balancing using CGR decomposition works correctly only if minor part atoms are lost 
# but multiplicity and formal charge are saved. In next release electronic state balansing will be added.

(<CGRtools.containers.molecule.MoleculeContainer at 0x11e545048>,
 <CGRtools.containers.molecule.MoleculeContainer at 0x11e5450e8>)

In [83]:
r1 # compare with initial reaction

## 1.4 Queries

CGRtools supports special objects for Queries. Queries are designed for substructure isomorphism. User can set number of neighbors and hybridization by himself (in molecules they could be calculated but could not be changed).

In [84]:
from CGRtools.containers import*

In [85]:
carb = m10.substructure([5,7,2]) # extract projection of carboxyl fragment
carb

CGRs also can be transformed into Query.

`QueryCGRContainer` is similar to QueryContainer class for CGRs and has the same API.

`QueryCGRContainer` take into account state of atoms and bonds in reactant and product, including neighbors and hybridization

In [86]:
# cgr_q = QueryCGRContainer(cgr1) # transfrom CGRContainer into QueryCGRContainer
# print(cgr_q)                    # print out signature of query

## 1.5. Molecules, CGRs, Reactions construction

CGRtools has API for objects construction from scratch.

CGR and Molecule has methods `add_atom` and `add_bond` for adding atoms and bonds.

In [87]:
from CGRtools.containers import *

In [88]:
m = MoleculeContainer() # new empty molecule

m.add_atom('C')  # add Carbon atom using element symbol
m.add_atom(6)    # add Carbon atom using element number. {'element': 6} is not valid, but {'element': 'O'} is also acceptable
m.add_atom('O', charge=-1) # add negatively charged Oxygen atom. Similarly other atomic properties can be set

# add_atom has second argument for setting atom number. 
# If not set, the next integer after the biggest among already created will be used.
m.add_atom('Na', charge=1, _map=4)
print(m)

[Na+].[OH-].C.C


In [89]:
m.add_bond(1, 2, 1) # add bond with order = 1 between atoms 1 and 2
m.add_bond(3, 2, bond=1) # the other possibility to set bond order

In [90]:
m.clean2d() #function to calculate atom coordinates.
m

Reactions can be constructed from molecules

In [91]:
r = ReactionContainer(reactants=[m1], products=[m11]) # one-step way to construct reaction
# or
r = ReactionContainer([m1], [m11]) # first list of MoleculeContainers is interpreted as reactants, second one - as products

In [92]:
r

In [93]:
# reactants, products, reagents attributes are tuples. 
try:
    r.products.append(m.copy()) # raise error for absent atom numbers
except AttributeError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-93-8650e408ca41>", line 3, in <module>
    r.products.append(m.copy()) # raise error for absent atom numbers
AttributeError: 'tuple' object has no attribute 'append'



In [94]:
r.fix_positions() # this method fixes coordinates of molecules in reaction
r

*QueryContainers* can be constructed in the same way as *MoleculeContainers*.

Unlike other containers *QueryContainers* additionally support atoms, neighbors and hybridization lists.

In [95]:
q = QueryContainer() # creation of empty container
q.add_atom('N')
q.add_atom('O') 
q.add_atom(atom='C', neighbors=[2, 3]) # add carbon atom, any isotope and multiplicity, neutral, 
                                                  # has 2 or 3 non-hydrogen heighbors
q.add_atom(atom='O') # add oxygen atom, any isotope and multiplicity, neutral, 
                             # number of neighbors and hybridization are irrelevant 
q.add_bond(1, 2, 1) # add single bond between atom 1 and 2 
q.add_bond(2, 3, 2) # add double bond between atom 1 and 2 
# any amide or carboxilic group will fit this query

In [96]:
print(q) # print out signature (SMILES-like)

# or
q.clean2d()
q

[C;23;]=[O][N].[O]


## 1.6. Extending CGRtools

You can easily customize CGRtools for your tasks.  
CGRtools is OOP-oriented library with subclassing and inheritance support.

As an example, we show how special marks on atoms for ligand donor centers can be added.

In [97]:
from CGRtools.containers import MoleculeContainer
from CGRtools.attributes import Atom

In [98]:
class MarkedAtom(Atom):  # this class will inherite Atom class
    __slots__ = '__mark' # all new attributes should be slotted!
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.__mark = None # set default value for added attribute

    @property
    def mark(self):        # created new property 
        return self.__mark
    
    @mark.setter
    def mark(self, mark):
        # do some checks and calculations
        self.__mark = mark

In [99]:
class MarkedMoleculeContainer(MoleculeContainer): # MarkedAtomContainer inherits MoleculeContainer
    node_attr_dict_factory = MarkedAtom # override atom container

In [100]:
m = MarkedMoleculeContainer() # create newly developed container MarkedMoleculeContainer
m.add_atom('C')               # add atom C using methods inherited from MoleculeContainer
m.add_atom('N')               # add atom N
m.add_bond(1, 2, 1)           # add bond
# doesn't work for the moment
# m.atom(2)['mark'] = 1  # set mark on atom. In this example dictinary setitem supported but update not.

In [101]:
# m.atom(2).mark # one can return mark